# 제주 한달살기 숙소 추천을 위한 분석
### 대상 - 펜션, 민박
- 이유 : 한달살기를 위한 시설이 준비된 숙소
- 호텔의 경우 가능 하겠지만 일반적인 한달살기 환경이 아니라는 가정

In [9]:
# !pip install selenium pandas bs4 xlrd requests tqdm

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

# 대기시간 설정을 위한 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from urllib.parse import quote
import time
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime, timedelta
from tqdm import tqdm

# 시스템 확인 - 파일 경로 확인
import os
# 다운로드 파일이동
import shutil

#엑셀파일 불러들이기 위한
import xlrd

#time.sleep을 위한 난수생성
import random

import pandas as pd

In [ ]:
# 야놀자 펜션/풀빌라 - https://www.yanolja.com/sub-home/pension
# 제주 지역 - https://www.yanolja.com/pension/r-900593?advert=AREA&topAdvertiseMore=1

url = 'https://www.yanolja.com/pension/r-900593?advert=AREA&topAdvertiseMore=1'
driver = webdriver.Chrome()
driver.get(url)


In [10]:
url = 'https://www.yanolja.com/pension/r-900593?advert=AREA&topAdvertiseMore=1'
driver = webdriver.Chrome()
driver.get(url)

for _ in range(10):
    last_height = driver.execute_script("return document.body.scrollHeight")

    # 창의 내리기 끝까지 하지않고, 중간에 멈춰서 한번은 page_down을 통해서
    driver.execute_script(f'window.scrollTo(0, {last_height - random.randint(1, 100)})')
    time.sleep(round(random.uniform(0.0, 1.0), 3))
    driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)     
    time.sleep(round(random.uniform(0.0, 1.2), 3))

    # 창의 높이가 달라지지 않으면 스크롤 내리기 중단
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break

In [12]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
soup

<html><head><link href="//yaimg.yanolja.com" rel="dns-prefetch"/><link href="https://yaimg.yanolja.com" rel="preconnect"/><link href="//yaimg.yanolja.com/joy/sunny/static/images/favicon/favicon_16x16.png" rel="shortcut icon"/><link href="//yaimg.yanolja.com/joy/sunny/static/images/favicon/favicon_16x16.png" rel="icon"/><link href="//yaimg.yanolja.com/joy/sunny/static/images/favicon/favicon_16x16.png" rel="icon" sizes="16x16"/><link href="//yaimg.yanolja.com/joy/sunny/static/images/favicon/favicon_32x32.png" rel="icon" sizes="32x32"/><link href="//yaimg.yanolja.com/joy/sunny/static/images/favicon/favicon_57x57.png" rel="icon" sizes="57x57"/><link href="//yaimg.yanolja.com/joy/sunny/static/images/favicon/favicon_72x72.png" rel="icon" sizes="72x72"/><link href="//yaimg.yanolja.com/joy/sunny/static/images/favicon/favicon_96x96.png" rel="icon" sizes="96x96"/><link href="//yaimg.yanolja.com/joy/sunny/static/images/favicon/favicon_120x120.png" rel="icon" sizes="120x120"/><link href="//yaimg.y

In [32]:
print('초이스', len(soup.select('.PlaceListItemBanner_container__ARsIm')))
print('프리미엄', len(soup.select('.PlaceListItemText_container__fUIgA')))


초이스 79
프리미엄 440


In [33]:
choice_list = soup.select('.PlaceListItemBanner_container__ARsIm')
choice = choice_list[1]
choice

<div class="PlaceListItemBanner_container__ARsIm banner-unit"><a href="/pension/10046493" title="제주 로비나발리플러스(신축,오션뷰)"><div class="PlaceListItemBanner_image__1-YJA"><div class="PlaceListImage_container__t-mGF"><div class="PlaceListImage_imageSmallPlaceholder__7ic0v" style='background-image: url("//yaimg.yanolja.com/joy/sunny/static/images/logo-yanolja-white.svg");'></div><div class="PlaceListImage_imageSmall__1NuLT" style='background-image: url("https://yaimg.yanolja.com/v5/2024/01/04/19/640/659701c5355989.97800178.jpg");'></div></div><div class="PlaceListDistance_containerBanner__2r0mD"><i></i><span>457.7km</span></div></div><div class="PlaceListItemBanner_contents__1oW8G"><div><div class="PlaceListTitle_container__qe7XH"><strong class="PlaceListTitle_text__2511B">제주 로비나발리플러스(신축,오션뷰)</strong></div><div class="PlaceListScore_container__2-JXJ PlaceListItemBanner_score__3HHN5"><span class="PlaceListScore_rating__3Glxf"><i class="PlaceListScore_star__2IZFX"></i>5.0</span><span class="Place

In [34]:
# choice 
choice_title = choice.select_one('.PlaceListTitle_text__2511B').text.strip()
choice_score = choice.select_one('.PlaceListScore_rating__3Glxf').text.strip()
choice_reviewNum = int(choice.select_one('.PlaceListScore_reviewInfo__3QSCU').text.strip()[1:-1])
choice_price = int(choice.select_one('.PlacePriceInfoV2_discountPrice__1PuwK').text.strip().replace(',', ''))
print(choice_title, choice_score, choice_reviewNum, choice_price)

제주 로비나발리플러스(신축,오션뷰) 5.0 145 230000


In [37]:
other_list = soup.select('.PlaceListItemText_container__fUIgA')
other = other_list[2]
other

<div class="PlaceListItemText_container__fUIgA text-unit"><a class="common_clearfix__M6urU" href="/pension/3008153" title="서귀포 산방산에펜션(개별바베큐,탄산온천반값할인권증정)"><div class="PlaceListItemText_area__3l67D"><div class="PlaceListItemText_image__2mlnK place-image"><div class="PlaceListImage_container__t-mGF"><div class="PlaceListImage_imageTextPlaceholder__1A9kl" style='background-image: url("//yaimg.yanolja.com/joy/sunny/static/images/logo-yanolja-white.svg");'></div><div class="PlaceListImage_imageText__2XEMn" style='background-image: url("https://yaimg.yanolja.com/v5/2022/11/28/11/1280/638498f7f3c168.72495912.jpg");'></div></div><div class="PlaceListDistance_containerText__2ODf3"><i></i><span>479.3km</span></div></div><div class="PlaceListItemText_contents__2GR73 place-content PlaceListItemText_singlePrice__1aj9I"><div class="ProductBadges_container__1A4Ml"><span class="ProductBadges_badge__PRhPM" style="color: rgb(24, 163, 77); border: 0.1rem solid rgba(24, 163, 77, 0.2); font-weight: 700;">야놀

In [38]:
other_title = other.select_one('.PlaceListTitle_text__2511B').text.strip()
other_score = other.select_one('.PlaceListScore_rating__3Glxf').text.strip()
other_reviewNum = int(other.select_one('.PlaceListScore_reviewInfo__3QSCU').text.strip()[1:-1])
other_price = int(other.select_one('.PlacePriceInfoV2_discountPrice__1PuwK').text.strip().replace(',', ''))
print(other_title, other_score, other_reviewNum, other_price)

# 각 영역에 해당하는 container만 다르고, 내부의 title, score, review, price 구조는 동일하네 

서귀포 산방산에펜션(개별바베큐,탄산온천반값할인권증정) 4.6 52 60000


In [44]:
other.select_one('a').attrs['href']

'/pension/3008153'

In [40]:
yanolja_list = [soup.select('.PlaceListItemBanner_container__ARsIm'), soup.select('.PlaceListItemText_container__fUIgA')]

for area_type in yanolja_list:
    print(len(area_type))

79
440


In [77]:
sub_url = 'https://place-site.yanolja.com/places/3005181'
driver = webdriver.Chrome()
driver.get(sub_url)

In [47]:
# 평점
driver.find_element(By.CLASS_NAME, 'address').text.strip()

'제주특별자치도 서귀포시 막숙포로 83'

In [78]:
# css-kav67y
driver.find_element(By.CLASS_NAME, 'css-14fkj0f').click()

In [69]:
# 소개 - 타이틀
driver.find_element(By.CSS_SELECTOR, '#BOTTOM_SHEET > div.css-gqqlqe > div.css-1ulzvpi > div > div:nth-child(1) > div > div.css-pw6xk > div > div:nth-child(1)').text


'솔라시도 펜션에 오신 것을 환영합니다.'

In [71]:
# 소개 - 내용
driver.find_element(By.CSS_SELECTOR, '#BOTTOM_SHEET > div.css-gqqlqe > div.css-1ulzvpi > div > div:nth-child(1) > div > div.css-pw6xk > div > div:nth-child(2)').text.strip()

'저희 솔라시도 펜션은 제주도 올레 7코스에 위치하여 아름다운 경관과 맑은 공기를 마음껏 즐기실 수 있습니다. 또한 아름다운 실내 인테리어와 차별화된 조식 서비스, 실내 카페 등 다양한 부대 서비스로 쾌적한 시간을 보내실 수 있습니다. 더불어 펜션 주변에는 아름다운 바다와 유명 관광지들이 즐비하여 각종 놀이시설과 즐길 거리들이 다양하여 가족, 단체, 연인 누구든지 즐거운 시간을 보내실 수 있습니다. 솔라시도 펜션은 오시는 분들께 즐거움과 깊이 간직할 수 있는 소중한 추억거리를 드릴 수 있도록 최선의 서비스를 제공하겠습니다. 감사합니다.'

In [72]:
# 편의시설 tag
driver.find_element(By.CSS_SELECTOR, '#BOTTOM_SHEET > div.css-gqqlqe > div.css-1ulzvpi > div > div:nth-child(2) > div > div.css-pw6xk > div.domestic_type.grid.css-bgisrg').text.strip()

'주차가능\n바베큐\n와이파이\n복층구조\n상비약'

In [75]:

# 편의시설 text
driver.find_element(By.CSS_SELECTOR, '#BOTTOM_SHEET > div.css-gqqlqe > div.css-1ulzvpi > div > div:nth-child(2) > div > div.css-pw6xk > div.domestic_type.plain.css-7kp13n').text.strip()

'복층\n· 아동,유아 동반 시 안전에 유의하시길 바랍니다.\nIPTV/WiFi\n· 객실에서 데이터 걱정 없이 빠르고 편리하게 인터넷을 즐겨보세요!\n· 사용 가능 객실 : 전 객실\n바비큐장\n· 사랑하는 사람과 맛있는 바비큐 파티를 즐겨 보세요!\n· 이용 시간 : 오후 1시 ~ 오후 9시까지\n· 이용 장소 : 야외 공용 바비큐장\n※ 바비큐기 웨버+숯+그릴 요금 : 20,000원 (2인~3인 기준) / +1인 5,000원\n※ 바비큐 SET : 30,000원 (1인 기준)\n· SET 항목 : 바비큐 시설 기본(바비큐기 웨버+숯+그릴) + 제주 생 돼지고기(400g 이상) + 전복(400g 이상) + 닥새우(400g 이상) 등 최소 3가지 이상, 쌈 야채, 구이용 채소, 기본 반찬, 양념장 등\n· 특화 바비큐 메뉴 : 전복, 비어치킨, 훈연 바비큐, 바비큐립 등 메뉴 주문형 바비큐 (비용은 펜션으로 별도문의 필요)\n· 예약 : 바비큐 시설 이용 시 최소 3시간 전에 예약, 바비큐 SET 항목은 이용 전일까지 예약\n※ 이용 시 유의사항\n· 기상 상황에 따라 예약은 취소될 수 있습니다. (펜션의 취소에 불구 고객님의 요청으로 준비될 경우 비용은 환불되지 않습니다.)\n· 바비큐 시설 이용 후 시설은 관리실에서, 가져오신 음식물은 이용 고객님이 정리하셔야 합니다.\n상비약\n· 제공 장소 : 관리실\n· 대여 시간: 제한 없음\n· 구비 항목 : 간단한 상비약\n· 구비 항목은 펜션 상황에 따라 변동될 수 있습니다.\n주차가능\n펜션즐기기\n[주변 관광지]\n· 중문 관광단지 : 12.36km\n· 중문 해수욕장 : 11.00km\n· 주상절리대 : 10.41km\n· 외돌개 : 4.26km\n· 범섬 : 608m\n· 상기 안내된 거리는 자동차 기준으로 표기된 점 참고 부탁드립니다.'

In [79]:
driver.find_element(By.CSS_SELECTOR, '#BOTTOM_SHEET > div.css-ex1z05 > img').click()


In [82]:
review_section = driver.find_element(By.CSS_SELECTOR, '#__next > div > div > main > article > div.css-c45a2y > div > div:nth-child(4) > div')

In [88]:
review_section.find_elements(By.CLASS_NAME, 'css-1eb9u5s')

#__next > div > div > main > article > div.css-c45a2y > div > div:nth-child(4) > div > div.css-1eb9u5s > span

StaleElementReferenceException: Message: stale element reference: stale element not found
  (Session info: chrome=120.0.6099.131); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF68C4A2142+3514994]
	(No symbol) [0x00007FF68C0C0CE2]
	(No symbol) [0x00007FF68BF676AA]
	(No symbol) [0x00007FF68BF779BF]
	(No symbol) [0x00007FF68BF6DB89]
	(No symbol) [0x00007FF68BF6BF22]
	(No symbol) [0x00007FF68BF6F38F]
	(No symbol) [0x00007FF68BF6F480]
	(No symbol) [0x00007FF68BFB1624]
	(No symbol) [0x00007FF68BFB197C]
	(No symbol) [0x00007FF68BFA68CC]
	(No symbol) [0x00007FF68BFD602F]
	(No symbol) [0x00007FF68BFA63B6]
	(No symbol) [0x00007FF68BFD6490]
	(No symbol) [0x00007FF68BFF28F6]
	(No symbol) [0x00007FF68BFD5D93]
	(No symbol) [0x00007FF68BFA4BDC]
	(No symbol) [0x00007FF68BFA5C64]
	GetHandleVerifier [0x00007FF68C4CE16B+3695259]
	GetHandleVerifier [0x00007FF6

In [84]:
review_section.find_element(By.TAG_NAME, 'a').click()

In [89]:
# review
review_url = sub_url + '/review'
driver = webdriver.Chrome()
driver.get(review_url)




In [112]:
review_list = driver.find_elements(By.CLASS_NAME, 'css-nkytaw')
review = review_list[46]

In [97]:
review

<selenium.webdriver.remote.webelement.WebElement (session="dce1e18c05edf830b56ea7b30339f5b0", element="FD6A704EFBA2EDC6E9EABC1275A2E114_element_22")>

In [114]:
# 리뷰내용
review.find_element(By.CLASS_NAME, 'css-djrif0').text.strip()

'.....'

In [98]:
review.find_elements(By.CSS_SELECTOR, 'div > div > div.css-176xgwq > div:nth-child(1) > div > div.css-ledymh > div > div ')

[<selenium.webdriver.remote.webelement.WebElement (session="dce1e18c05edf830b56ea7b30339f5b0", element="FD6A704EFBA2EDC6E9EABC1275A2E114_element_481")>]

In [ ]:
#__next > div > div > main > div > div:nth-child(4) > div:nth-child(2) > div:nth-child(17) > div > div > div.css-176xgwq > div:nth-child(1) > div > div.css-ledymh > div > div > svg:nth-child(1) > path

In [105]:
driver.find_element(By.CSS_SELECTOR, '#__next > div > div > main > div > div:nth-child(4) > div:nth-child(2) > div:nth-child(17) > div > div > div.css-176xgwq > div:nth-child(1) > div > div.css-ledymh > div > div > svg:nth-child(1) > path').get_attribute('fill')


'#fdbd00'

In [115]:
for star_num in range(1, 6):
    print(driver.find_element(By.CSS_SELECTOR, f'#__next > div > div > main > div > div:nth-child(4) > div:nth-child(2) > div:nth-child(17) > div > div > div.css-176xgwq > div:nth-child(1) > div > div.css-ledymh > div > div > svg:nth-child({star_num}) > path').get_attribute('fill'))

#fdbd00
#fdbd00
#fdbd00
#fdbd00
#fdbd00


In [ ]:
#fdbd00 노란색
#ffffff 흰색

#__next > div > div > main > div > div:nth-child(4) > div:nth-child(3) > div:nth-child(7) > div > div > div.css-176xgwq > div:nth-child(1) > div > div.css-ledymh > div > div > svg:nth-child(2) > path
#__next > div > div > main > div > div:nth-child(4) > div:nth-child(3) > div:nth-child(7) > div > div > div.css-176xgwq > div:nth-child(1) > div > div.css-ledymh > div > div > svg:nth-child(1) > path

In [127]:
print(driver.find_element(By.CSS_SELECTOR, '#__next > div > div > main > div > div:nth-child(4) > div:nth-child(3) > div:nth-child(7) > div > div > div.css-176xgwq > div:nth-child(1) > div > div.css-ledymh > div > div > svg:nth-child(1) > path').get_attribute('fill'))

#fdbd00


In [ ]:
#__next > div > div > main > div > div:nth-child(4) > div:nth-child(3) > div:nth-child(7)

In [128]:
review.find_element(By.CSS_SELECTOR, 'div > div > div.css-176xgwq > div:nth-child(1) > div > div.css-ledymh > div > div > svg:nth-child(1) > path').get_attribute('fill')

'#fdbd00'

In [129]:
review.find_element(By.CSS_SELECTOR, 'div > div > div.css-176xgwq > div:nth-child(1) > div > div.css-ledymh > div > div > svg:nth-child(2) > path').get_attribute('fill')

'#ffffff'

In [130]:
for star_num in range(1, 6):
    print(review.find_element(By.CSS_SELECTOR, f'div > div > div.css-176xgwq > div:nth-child(1) > div > div.css-ledymh > div > div > svg:nth-child({star_num}) > path').get_attribute('fill'))

#fdbd00
#ffffff
#ffffff
#ffffff
#ffffff


In [131]:
star_list = []
for star_num in range(1, 6):
    star_color = review.find_element(By.CSS_SELECTOR, f'div > div > div.css-176xgwq > div:nth-child(1) > div > div.css-ledymh > div > div > svg:nth-child({star_num}) > path').get_attribute('fill')
    star_list.append(star_color)
star_list


['#fdbd00', '#ffffff', '#ffffff', '#ffffff', '#ffffff']

In [132]:
#노란색 갯수가 평점!!!
star_list.count('#fdbd00') 

1

In [145]:
### 리뷰 내용정리
# review
review_url = sub_url + '/review'
driver = webdriver.Chrome()
driver.get(review_url)

review_list = driver.find_elements(By.CLASS_NAME, 'css-nkytaw')

temp = pd.DataFrame(columns = ['star', 'text'])

for review in review_list[:5]:
    review_temp = []
    
    # 리뷰내용
    review_text = review.find_element(By.CLASS_NAME, 'css-djrif0').text.strip()
    review_text = re.sub('[-=+,/\?:^.@*\"※~ㆍ!』><‘|\(\)\[\]`\'…》\”\“\’·]', "", review_text)

    # 별점
    star_list = []
    for star_num in range(1, 6):
        star_color = review.find_element(By.CSS_SELECTOR, f'div > div > div.css-176xgwq > div:nth-child(1) > div > div.css-ledymh > div > div > svg:nth-child({star_num}) > path').get_attribute('fill')
        star_list.append(star_color)

    star_score = star_list.count('#fdbd00')  #노란색 갯수가 평점!!!
    time.sleep(round(random.uniform(0.0, 1.2), 3))

    review_temp.append({'star' : star_score,
                        'text' : review_text
                        })
    review_temp = pd.DataFrame(review_temp)

    temp = pd.concat([temp, review_temp])
    time.sleep(round(random.uniform(0.0, 1.2), 3))    

,star,text


In [141]:

temp = pd.DataFrame(columns = ['star', 'text'])
temp

,star,text
